<a href="https://colab.research.google.com/github/Nadian-Ali/Temporary_Vid_classification/blob/main/CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd '/content/drive/My Drive/Projects/Video_Rec/' 

/content/drive/My Drive/Projects/Video_Rec


In [2]:
!pip install -q git+https://github.com/tensorflow/docs

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow_docs.vis import embed
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import cv2
import os

<h4>Load data</h4>

In [8]:
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1024
IMG_SIZE = 128
EPOCHS = 20

In [7]:
train_data   = np.load('resampled_train_dataset.npy')
val_data     = np.load('resampled_val_dataset.npy')
train_labels = np.load('train_labels.npy')
val_labels   = np.load('val_labels.npy')


In [41]:
# Utility for our sequence model.
def get_sequence_model():
    # class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    # mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.7)(x)
    # x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(1, activation="sigmoid")(x)

    rnn_model = keras.Model(frame_features_input, output)

    rnn_model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
        metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
# def run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )


#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# _, sequence_model = run_experiment()

In [42]:
    seq_model = get_sequence_model()
    history = seq_model.fit(
        train_data,
        train_labels,
        # validation_split=0.3,
        batch_size = 4,
        shuffle = True,
        epochs=30,
        validation_data=(val_data,val_labels),
        # callbacks=[checkpoint],
    )


Epoch 1/30
4/4 [==============================] - 4s 280ms/step - loss: 0.6402 - accuracy: 0.5385 - val_loss: 0.7694 - val_accuracy: 0.5000
Epoch 2/30
4/4 [==============================] - 0s 28ms/step - loss: 0.7556 - accuracy: 0.4615 - val_loss: 0.7660 - val_accuracy: 0.5000
Epoch 3/30
4/4 [==============================] - 0s 28ms/step - loss: 0.8920 - accuracy: 0.3846 - val_loss: 0.7735 - val_accuracy: 0.6667
Epoch 4/30
4/4 [==============================] - 0s 28ms/step - loss: 0.8288 - accuracy: 0.4615 - val_loss: 0.7685 - val_accuracy: 0.6667
Epoch 5/30
4/4 [==============================] - 0s 28ms/step - loss: 0.8654 - accuracy: 0.4615 - val_loss: 0.7628 - val_accuracy: 0.6667
Epoch 6/30
4/4 [==============================] - 0s 26ms/step - loss: 0.8518 - accuracy: 0.4615 - val_loss: 0.7449 - val_accuracy: 0.5000
Epoch 7/30
4/4 [==============================] - 0s 27ms/step - loss: 0.5617 - accuracy: 0.8462 - val_loss: 0.7477 - val_accuracy: 0.5000
Epoch 8/30
4/4 [==========